In [1]:
import re
import csv
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from os import listdir
from datatable import dt, f, by
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from utils import get_path, data_load, group_cols

# Descrição

Entre os desafios mais urgentes do Sistema Único de Saúde (SUS) estão a ampliação do acesso e a adequação dos serviços ofertados. No Brasil, as relações existentes entre regiões e redes de atenção à saúde indicam que o avanço do processo de regionalização do SUS pode interferir positivamente na oferta de ações de serviços de saúde, tornando-a mais eficiente e abrangente.

[Saiba mais](https://www.resbr.net.br/apresentacao/)

[Plataforma de acesso ao Banco de Indicadores](https://indicadores.resbr.net.br/view/)

# Fontes de dados

## SIH

### Colunas da tabela

In [2]:
cols_sih = [
  'cnes',
]

In [3]:
path_sih = get_path('SIH', 'sih.jay')
df_sih = dt.fread(path_sih)
df_sih = df_sih[:, cols_sih]
df_sih

,cnes
,▪▪▪▪
0,2001020
1,2000733
2,2000733
3,2000733
4,2000733
5,2000733
6,2000733
7,2001020
8,2000733


### Tabela agrupada

In [4]:
df_sih = group_cols(df_sih, cols_sih).sort(-f.count)
df_sih

,cnes,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2232,101955
1,3151794,75209
2,2323397,74712
3,2311682,64833
4,26794,63700
5,2020068,59613
6,6627595,58139
7,2799278,57750
8,10480,57201


# Web Scrap

In [5]:
def get_cnes_row(driver, url, cnes):
  driver.get(url + cnes)
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  dict_cols = dict()
  columns = soup.find_all('td')
  for col in columns[:-1]:
    c_names = re.findall("'([^']+)'", col['data-title'])
    c_name = c_names[0] if len(c_names) == 1 else c_names[1] # {1: 'Nome Fantasia', 2: 'Nome Empresarial'}
    value = re.findall("\\t{3}(.+)\\n", col.contents[0])
    dict_cols[c_name] = value
  df = pd.DataFrame.from_dict(dict_cols)
  return df

In [6]:
options = Options()
options.add_argument('--headless')
chromedriver = '/Users/lucas/Desktop/chromedriver'
driver = webdriver.Chrome(chromedriver, options=options)

/var/folders/9d/972tfrtd01z2hhl836z97zdr0000gn/T/ipykernel_58679/1031672721.py:4: DeprecationWarning:

executable_path has been deprecated, please pass in a Service object



In [ ]:
url = 'http://cnes.datasus.gov.br/pages/estabelecimentos/consulta.jsp?search='

In [7]:
cnes_list = [('0' * 7 + str(c))[-7:] for c in df_sih['cnes'].to_list()[0]]
dfs = [get_cnes_row(driver, url, cnes) for cnes in tqdm(cnes_list)]
len(dfs)

100%|██████████| 4150/4150 [14:17<00:00,  4.84it/s]


4150

100%|██████████| 4150/4150 [14:17<00:00,  4.84it/s]

# Gerando tabela

## CNES coletados

In [8]:
df_cnes_collect = pd.DataFrame()
df_cnes_collect['cnes'] = cnes_list
df_cnes_collect['collected'] = [len(df) ==  1 for df in dfs]
df_cnes_collect

,cnes,collected
0,0002232,True
1,3151794,True
2,2323397,True
3,2311682,True
4,0026794,True
...,...,...
4145,7320175,True
4146,7603029,True
4147,7621442,True
4148,7704364,True


In [18]:
n_cnes = df_cnes_collect.shape[0]
n_collected = df_cnes_collect[df_cnes_collect['collected'] == True].shape[0]
percent_clct = n_collected / n_cnes
print(f'Collected {n_collected} cnes from {n_cnes} ({percent_clct:.2f}%)')

Collected 3928 cnes from 4150 (0.95%)


## Concatenando tabelas

Concatenação das tabelas em uma

In [9]:
df_cnes = pd.concat([df for df in dfs if len(df) ==  1])
df_cnes

,UF,Município,CNES,Nome Fantasia,Natureza Jurídica(Grupo),Gestão,Atende SUS
0,SE,ARACAJU,0002232,HOSPITAL E MATERNIDADE SANTA IZABEL,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
0,AM,MANAUS,3151794,MATERNIDADE DE REFERENCIA ANA BRAGA,ADMINISTRAÇÃO PÚBLICA,E,SIM
0,PI,TERESINA,2323397,MATERNIDADE DONA EVANGELINA ROSA,ADMINISTRAÇÃO PÚBLICA,M,SIM
0,MT,CUIABA,2311682,HOSPITAL SANTA HELENA,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
0,MG,BELO HORIZONTE,0026794,HOSPITAL SOFIA FELDMAN,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
...,...,...,...,...,...,...,...
0,SP,SAO MIGUEL ARCANJO,7320175,HOSPITAL SAO MIGUEL ARCANJO,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
0,MA,BOA VISTA DO GURUPI,7603029,HOSPITAL MUNICIPAL LENILDE OLIVEIRA,ADMINISTRAÇÃO PÚBLICA,M,SIM
0,ES,VITORIA,7621442,HOSPITAL ESTADUAL DE URGENCIA E EMERGENCIA,ADMINISTRAÇÃO PÚBLICA,E,SIM
0,RO,CACOAL,7704364,HOSPITAL DE URGENCIA E EMERGENCIA REGIONAL DE ...,ADMINISTRAÇÃO PÚBLICA,E,SIM


## CNES não atendem SUS

### Quantidade dos que atendem

In [19]:
df_cnes['Atende SUS'].value_counts()

SIM    3814
NÃO     114
Name: Atende SUS, dtype: int64

### Os que não antendem

In [27]:
df_not_sus = df_cnes[df_cnes['Atende SUS'] == 'NÃO']
df_not_sus =  df_not_sus.sort_values(by='UF')
df_not_sus

,UF,Município,CNES,Nome Fantasia,Natureza Jurídica(Grupo),Gestão,Atende SUS
0,AL,MACEIO,2808889,HOSPITAL SAO RAFAEL,ENTIDADES EMPRESARIAIS,M,NÃO
0,AL,MACEIO,6782477,HOSPITAL GERAL E MATERNIDADE ALERTA MEDICO,ENTIDADES EMPRESARIAIS,M,NÃO
0,BA,IBIPEBA,2412985,HOSPITAL BOM JESUS DE NAZARE,ENTIDADES EMPRESARIAIS,E,NÃO
0,BA,IBIRATAIA,2413051,MATERNIDADE AURELINA VIRGILIA FAIR,ENTIDADES SEM FINS LUCRATIVOS,M,NÃO
0,BA,EUNAPOLIS,2556642,HOSPITAL E MATERNIDADE DE EUNAPOLIS,ENTIDADES EMPRESARIAIS,M,NÃO
...,...,...,...,...,...,...,...
0,SP,MORUNGABA,2026392,HOSPITAL SANTO ANTONIO,ENTIDADES EMPRESARIAIS,M,NÃO
0,SP,CAIEIRAS,6817203,HOSPITAL DE CLINICAS CAIEIRAS,ENTIDADES EMPRESARIAIS,M,NÃO
0,SP,SAO PAULO,2076934,HOSPITAL SOROCABANA,ENTIDADES SEM FINS LUCRATIVOS,M,NÃO
0,SP,TUPA,2080672,HOSPITAL SAO FRANCISCO TUPA,ENTIDADES SEM FINS LUCRATIVOS,M,NÃO


### UF

In [26]:
df_not_sus['UF'].value_counts()

PA    18
BA    15
PR    14
GO    10
RJ     8
PE     8
CE     6
SP     5
MT     5
MS     4
MA     4
MG     4
PI     3
ES     2
RN     2
AL     2
SE     1
RS     1
PB     1
DF     1
Name: UF, dtype: int64

### Natureza Jurídica(Grupo)

In [23]:
df_not_sus['Natureza Jurídica(Grupo)'].value_counts()

ENTIDADES EMPRESARIAIS           95
ENTIDADES SEM FINS LUCRATIVOS    17
ADMINISTRAÇÃO PÚBLICA             2
Name: Natureza Jurídica(Grupo), dtype: int64

# Exportando tabela

In [10]:
path_cnes = get_path('LOCALIDADES', 'cnes.csv.gzip')
df_cnes.to_csv(path_cnes, index=False)

In [11]:
pd.read_csv(path_cnes, low_memory=False)

,UF,Município,CNES,Nome Fantasia,Natureza Jurídica(Grupo),Gestão,Atende SUS
0,SE,ARACAJU,2232,HOSPITAL E MATERNIDADE SANTA IZABEL,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
1,AM,MANAUS,3151794,MATERNIDADE DE REFERENCIA ANA BRAGA,ADMINISTRAÇÃO PÚBLICA,E,SIM
2,PI,TERESINA,2323397,MATERNIDADE DONA EVANGELINA ROSA,ADMINISTRAÇÃO PÚBLICA,M,SIM
3,MT,CUIABA,2311682,HOSPITAL SANTA HELENA,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
4,MG,BELO HORIZONTE,26794,HOSPITAL SOFIA FELDMAN,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
...,...,...,...,...,...,...,...
3923,SP,SAO MIGUEL ARCANJO,7320175,HOSPITAL SAO MIGUEL ARCANJO,ENTIDADES SEM FINS LUCRATIVOS,M,SIM
3924,MA,BOA VISTA DO GURUPI,7603029,HOSPITAL MUNICIPAL LENILDE OLIVEIRA,ADMINISTRAÇÃO PÚBLICA,M,SIM
3925,ES,VITORIA,7621442,HOSPITAL ESTADUAL DE URGENCIA E EMERGENCIA,ADMINISTRAÇÃO PÚBLICA,E,SIM
3926,RO,CACOAL,7704364,HOSPITAL DE URGENCIA E EMERGENCIA REGIONAL DE ...,ADMINISTRAÇÃO PÚBLICA,E,SIM
